# University of Illinois Chicago (UIC) Current Conditions

In [5]:
from IPython.display import IFrame
import pandas as pd
import sage_data_client
import holoviews as hv
import hvplot.pandas
from metpy.units import units
import metpy.calc as mpcalc
from bokeh.models import DatetimeTickFormatter
import sage_data_client

hv.extension("bokeh")

def apply_formatter(plot, element):
    plot.handles['xaxis'].formatter = DatetimeTickFormatter(hours='%m/%d/%Y \n %l:%M %p',
                                                            minutes='%m/%d/%Y \n %l:%M %p',
                                                            hourmin='%m/%d/%Y \n %l:%M %p',
                                                            days='%m/%d/%Y \n %l:%M %p',
                                                            months='%m/%d/%Y \n %l:%M %p')

def add_local_time(df):
    df["Local_Time"] = pd.to_datetime(df.index,unit='s', utc=True)\
                          .map(lambda x: x.tz_convert('America/Chicago'))
    return df

def get_field_convert_to_degF(df, variable):
    subset = df.loc[df.name == variable]

    subset["value"] = (subset.value.values * units.degC).to("degF")
    return subset.value

def get_field(df, variable):
    subset = df.loc[df.name == variable]
    return subset.value

import warnings
warnings.filterwarnings('ignore')

## Temperature and Dewpoint

In [2]:
wxt_df = sage_data_client.query(
    start="-12h", 
    filter={
        "vsn": "W096",
        "name": "wxt.env.temp|wxt.env.humidity"
    }
).set_index("timestamp")

temp_df = wxt_df.loc[wxt_df.name == "wxt.env.temp"]
hum_df = wxt_df.loc[wxt_df.name == "wxt.env.humidity"]

min_df = temp_df[["value"]].resample("1min").mean()
min_df["rh"] = hum_df[["value"]].resample("1min").mean()

min_df["tmpf"] = (min_df.value.values * units.degC).to("degF").m
min_df["dwpf"] = mpcalc.dewpoint_from_relative_humidity(min_df.tmpf.values * units.degF,
                                       min_df.rh.values * units.percent).to("degF").m

min_df = add_local_time(min_df)

temp_plot = min_df.hvplot(x='Local_Time',
                          y='tmpf',
                          color = "r",
                          label="Temperature [degF]"
                         )
dewp_plot = min_df.hvplot(x='Local_Time',
                          y='dwpf',
                          color="g",
                          title="Current Weather at the University of Illinois Chicago (UIC)",
                          label="Dewpoint [degF]",
                          ylabel="Temperature | Dewpoint \n [degF]",
                          xlabel="Local Time (Central/US)")

(temp_plot * dewp_plot).opts(hooks=[apply_formatter], legend_position='right')

:Overlay
   .Curve.Temperature_left_square_bracket_degF_right_square_bracket :Curve   [Local_Time]   (tmpf)
   .Curve.Dewpoint_left_square_bracket_degF_right_square_bracket    :Curve   [Local_Time]   (dwpf)

## Air Quality

In [3]:
df = sage_data_client.query(
    start="-12h", 
    filter={
        "vsn": "W095",
        "name": "aqt.particle.pm2.5"
    }
).set_index("timestamp")

df = add_local_time(df)[["value", "Local_Time"]].resample("5min").mean()

df.hvplot(label='PM 2.5 Values at the University of Illinois Chicago (UIC)',
          x='Local_Time',
          y='value',
          ylabel='PM 2.5 \n [microgram per cubic meter]',
          xlabel="Local Time (Central/US)").opts(hooks=[apply_formatter])

:Curve   [Local_Time]   (value)

## Soil Temperature

In [4]:
df = sage_data_client.query(
    start="-1d", 
    filter={
        "vsn": "W096",
        "name": "temp_d1|temp_d2|temp_d3|temp_d4",
        "deviceName": "MFR node MNLA4O106"
    }
).set_index("timestamp")

soil_temp_df = pd.DataFrame({"~6 inches below the surface": get_field_convert_to_degF(df, "temp_d1"),
                             "~1 feet below the surface": get_field_convert_to_degF(df, "temp_d2"),
                             "~1.5 feet below the surface": get_field_convert_to_degF(df, "temp_d3"),
                             "~2 feet below the surface": get_field_convert_to_degF(df, "temp_d4")})

soil_temp_df = add_local_time(soil_temp_df)

first = soil_temp_df.hvplot(ylabel="Soil Temperature [degF]",
                    xlabel="Time/Date",
                    x='Local_Time',
                    title="Soil Temperature at University of Illinois Chicago \n Parking Lot 5").opts(hooks=[apply_formatter])

df = sage_data_client.query(
    start="-1d", 
    filter={
        "vsn": "W096",
        "name": "temp_d1|temp_d2|temp_d3|temp_d4",
        "deviceName": "MFR node MNLA4O105"
    }
).set_index("timestamp")

soil_temp_df = pd.DataFrame({"~6 inches below the surface": get_field_convert_to_degF(df, "temp_d1"),
                             "~1 feet below the surface": get_field_convert_to_degF(df, "temp_d2"),
                             "~1.5 feet below the surface": get_field_convert_to_degF(df, "temp_d3"),
                             "~2 feet below the surface": get_field_convert_to_degF(df, "temp_d4")})

soil_temp_df = add_local_time(soil_temp_df)

second = soil_temp_df.hvplot(ylabel="Soil Temperature [degF]",
                    xlabel="Time/Date",
                    x='Local_Time',
                    title="Soil Temperature at University of Illinois Chicago \n Greenhouse").opts(hooks=[apply_formatter])

(first + second).cols(1)

:Layout
   .NdOverlay.I  :NdOverlay   [Variable]
      :Curve   [Local_Time]   (value)
   .NdOverlay.II :NdOverlay   [Variable]
      :Curve   [Local_Time]   (value)

## Soil Moisture

In [6]:
df = sage_data_client.query(
    start="-1d", 
    filter={
        "vsn": "W096",
        "name": "vwc_d1|vwc_d2|vwc_d3|vwc_d4",
        "deviceName": "MFR node MNLA4O106"
    }
).set_index("timestamp")

soil_temp_df = pd.DataFrame({"~6 inches below the surface": get_field(df, "vwc_d1"),
                             "~1 feet below the surface": get_field(df, "vwc_d2"),
                             "~1.5 feet below the surface": get_field(df, "vwc_d3"),
                             "~2 feet below the surface": get_field(df, "vwc_d4")})

soil_temp_df = add_local_time(soil_temp_df)

first = soil_temp_df.hvplot(ylabel="Soil Moisture [%]",
                    xlabel="Time/Date",
                    x='Local_Time',
                    title="Soil Moisture at University of Illinois Chicago \n Parking Lot 5").opts(hooks=[apply_formatter])

df = sage_data_client.query(
    start="-1d", 
    filter={
        "vsn": "W096",
        "name": "vwc_d1|vwc_d2|vwc_d3|vwc_d4",
        "deviceName": "MFR node MNLA4O105"
    }
).set_index("timestamp")

soil_temp_df = pd.DataFrame({"~6 inches below the surface": get_field(df, "vwc_d1"),
                             "~1 feet below the surface": get_field(df, "vwc_d2"),
                             "~1.5 feet below the surface": get_field(df, "vwc_d3"),
                             "~2 feet below the surface": get_field(df, "vwc_d4")})

soil_temp_df = add_local_time(soil_temp_df)

second = soil_temp_df.hvplot(ylabel="Soil Moisture [%]",
                    xlabel="Time/Date",
                    x='Local_Time',
                    title="Soil Moisture at University of Illinois Chicago \n Greenhouse").opts(hooks=[apply_formatter])

(first + second).cols(1)

:Layout
   .NdOverlay.I  :NdOverlay   [Variable]
      :Curve   [Local_Time]   (value)
   .NdOverlay.II :NdOverlay   [Variable]
      :Curve   [Local_Time]   (value)

## Water Depth

In [11]:
df = sage_data_client.query(
    start="-1d", 
    filter={
        "vsn": "W096",
        "name": "water_depth",
        "deviceName": "MFR node MNLA4O106"
    }
).set_index("timestamp")

df = add_local_time(df)

# Convert from mm to meters
df["value"] = df["value"]/1000.

first = df.hvplot(x='Local_Time',
          ylabel='Ground Water Height [meters]',
          xlabel="Date | Time",
          title="Ground Water Height at University of Illinois Chicago \n Parking Lot 5").opts(hooks=[apply_formatter])

# df = sage_data_client.query(
#    start="-1d", 
#    filter={
#        "vsn": "W096",
#        "name": "water_depth",
#        "deviceName": "MFR node MNLA4O105"
#    }
#).set_index("timestamp")

#df = add_local_time(df)

# Convert from mm to meters
#df["value"] = df["value"]/1000.

#second = df.hvplot(x='Local_Time',
#         ylabel='Ground Water Height [meters]',
#          xlabel="Date | Time",
#          title="Ground Water Height at University of Illinois Chicago \n Greenhouse").opts(hooks=[apply_formatter])

first

:Curve   [Local_Time]   (value)

## Location
Observations from the University of Illinois Chicago (UIC) campus, near the greenhouse.

<div>
<img src="../../images/uic-tower.jpg" width="500"/>
</div>

### Greenhouse (Weather, Air Quality, Soil)

In [8]:
IFrame("https://www.google.com/maps/embed?pb=!1m17!1m12!1m3!1d2963.349820064459!2d-87.64835808836337!3d41.86947217112274!2m3!1f0!2f0!3f0!3m2!1i1024!2i768!4f13.1!3m2!1m1!2zNDHCsDUyJzEwLjEiTiA4N8KwMzgnNDQuOCJX!5e1!3m2!1sen!2sus!4v1745335739504!5m2!1sen!2sus", width=700, height=350)

### Parking Lot 5 (Soil, Water Depth)

In [9]:
IFrame("https://www.google.com/maps/embed?pb=!1m17!1m12!1m3!1d2963.400217962413!2d-87.65168628836342!3d41.86838497112295!2m3!1f0!2f0!3f0!3m2!1i1024!2i768!4f13.1!3m2!1m1!2zNDHCsDUyJzA2LjIiTiA4N8KwMzgnNTYuOCJX!5e1!3m2!1sen!2sus!4v1745335901657!5m2!1sen!2sus", width=700, height=350)